# 19.05 Evaluating Performance
## Assignment

###  2. House prices model

In this exercise, you'll work on your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

* Load the **houseprices** data from Thinkful's database.
* Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.
* Do you think your model is satisfactory? If so, why?
* In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables. 
* For each model you try, get the goodness of fit metrics and compare your models with each other. Which model is the best and why?

#### _First, load the dataset from the **weatherinszeged** table from Thinkful's database._

In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import statsmodels.api as sm

from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

pd.options.display.float_format = "{:3f}".format

warnings.filterwarnings(action="ignore")

kagle = dict(
    drivername = "postgresql",
    username = "dsbc_student",
    password = "7*.8G9QH21",
    host = "142.93.121.174",
    port = "5432",
    database = "houseprices"
)

In [2]:
# Load the data from the "houseprices" database
engine=create_engine(URL(**kagle), echo=True)

houses_raw=pd.read_sql_query("SELECT * FROM houseprices", con=engine)

engine.dispose()

2020-01-06 20:45:01,446 INFO sqlalchemy.engine.base.Engine select version()
2020-01-06 20:45:01,454 INFO sqlalchemy.engine.base.Engine {}
2020-01-06 20:45:01,566 INFO sqlalchemy.engine.base.Engine select current_schema()
2020-01-06 20:45:01,567 INFO sqlalchemy.engine.base.Engine {}
2020-01-06 20:45:01,671 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-01-06 20:45:01,674 INFO sqlalchemy.engine.base.Engine {}
2020-01-06 20:45:01,729 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-01-06 20:45:01,731 INFO sqlalchemy.engine.base.Engine {}
2020-01-06 20:45:01,784 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2020-01-06 20:45:01,786 INFO sqlalchemy.engine.base.Engine {}
2020-01-06 20:45:01,889 INFO sqlalchemy.engine.base.Engine SELECT * FROM houseprices
2020-01-06 20:45:01,891 INFO sqlalchemy.engine.base.Engine {}


In [3]:
houses_working = houses_raw.copy()

Select a subset of variables on which to base the model on 

In [4]:
houses_df = houses_working[["neighborhood","overallqual","lotarea",
                            "totalbsmtsf","firstflrsf","grlivarea",
                            "totrmsabvgrd","garagecars","saleprice"]]

In [5]:
houses_df.describe()

,overallqual,lotarea,totalbsmtsf,firstflrsf,grlivarea,totrmsabvgrd,garagecars,saleprice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,6.099315,10516.828082,1057.429452,1162.626712,1515.463699,6.517808,1.767123,180921.195890
std,1.382997,9981.264932,438.705324,386.587738,525.480383,1.625393,0.747315,79442.502883
min,1.000000,1300.000000,0.000000,334.000000,334.000000,2.000000,0.000000,34900.000000
25%,5.000000,7553.500000,795.750000,882.000000,1129.500000,5.000000,1.000000,129975.000000
50%,6.000000,9478.500000,991.500000,1087.000000,1464.000000,6.000000,2.000000,163000.000000
75%,7.000000,11601.500000,1298.250000,1391.250000,1776.750000,7.000000,2.000000,214000.000000
max,10.000000,215245.000000,6110.000000,4692.000000,5642.000000,14.000000,4.000000,755000.000000


In [6]:
houses_df.describe(include=["O"])

,neighborhood
count,1460
unique,25
top,NAmes
freq,225


In [7]:
# Create a set of dummies for the neighborhood variable, prefix the dummies with "neighborhood"
houses_df = pd.concat([houses_df, pd.get_dummies(houses_df["neighborhood"], prefix="neighborhood",drop_first=True)], axis=1)

# Create a set of dumies for the overallqual variable, previs the dummies with "overallqual"
houses_df = pd.concat([houses_df, pd.get_dummies(houses_df["overallqual"], prefix="overallqual",drop_first=True)], axis=1)

In [8]:
# Get a list of column names to be used for feature consideration
feature_names = houses_df.iloc[:,2:].columns.to_list()

# Pop saleprice from the list of feature_names
feature_names.pop(6)

'saleprice'

In [9]:
# Y is the target variable
Y = houses_df["saleprice"]

# X is the feature set
X = houses_df[feature_names]

# Add a constant to the model
X = sm.add_constant(X)

# Fit an OLS model using statsmodel
results = sm.OLS(Y,X).fit()

# Print the results
print(results.summary())

# Tear out the columns that I'm intersted in comparing 
first_model = results.summary2().tables[1]
first_model = first_model[["Coef.","P>|t|"]].round(4)

                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.834
Model:                            OLS   Adj. R-squared:                  0.829
Method:                 Least Squares   F-statistic:                     182.7
Date:                Mon, 06 Jan 2020   Prob (F-statistic):               0.00
Time:                        20:48:18   Log-Likelihood:                -17234.
No. Observations:                1460   AIC:                         3.455e+04
Df Residuals:                    1420   BIC:                         3.476e+04
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                  2.73e+04 

#### _Do you think your model is satisfactory? If so, why?_

I beleieve that the model is fairly well performant.  The R-squared and adjusted R-squared are 83.4% and 82.9% respectively and both have a fairly low AIC and BIC score at 3.455 and 3.476 respectively.

#### _In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables._

